In [ ]:
# Requirements:
# pip install wetterdienst geopy pandas

import pandas as pd
from geopy.geocoders import Nominatim
from wetterdienst.provider.dwd.observation import (
    DwdObservationRequest,
    DwdObservationMetadata
)
from wetterdienst.metadata.parameter import Parameter as DwdPara
from wetterdienst.metadata.resolution import Resolution as DwdRes
from wetterdienst.metadata.period import Period as DwdPeriod

import datetime as dt

In [ ]:
params = [
    # ("hourly", "sun"),
    # ("daily", "solar"),
    ("10_minutes", "solar", "radiation_global"),
    ("10_minutes", "solar", "sunshine_duration"),
    # ("hourly", "solar", "radiation_global"),
    # ("hourly", "solar", "sunshine_duration"),
]

start_date = dt.datetime(2020, 4, 1)
end_date = dt.datetime(2025, 9, 1)

station_ids = []

all_data = {}

for period in ("historical", "recent"):
    req = DwdObservationRequest(
        parameters=params,
        start_date=start_date,
        end_date=end_date,
        periods=period,
    )

    stations = req.filter_by_station_id(station_id=station_ids)

    stations_df = stations.df.to_pandas()
    value_df = stations.values.all().df.to_pandas()

    all_data[period] = stations_df[
        ["resolution", "dataset", "station_id", "name"]
    ].merge(value_df, on=["station_id", "resolution", "dataset"], how="left")


In [ ]:
display(all_data["historical"].groupby(["resolution", "dataset", "station_id", "parameter"]).agg({"date": ["min", "max"]}))
display(all_data["recent"].groupby(["resolution", "dataset", "station_id", "parameter"]).agg({"date": ["min", "max"]}))

In [ ]:
# add all data from recent thats not in historical
# prefer historical has higher quality assurance